- https://google.github.io/adk-docs/deploy/agent-engine/
- https://cloud.google.com/python/docs/reference/vertexai/latest/vertexai.preview.reasoning_engines.ReasoningEngine#vertexai_preview_reasoning_engines_ReasoningEngine_list

# install packages

In [2]:
!pip install -U google-genai google-adk google-cloud-aiplatform

  Using cached google_adk-1.1.1-py3-none-any.whl.metadata (9.5 kB)
Using cached google_adk-1.1.1-py3-none-any.whl (1.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 96.0 MB/s eta 0:00:00
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.16.1
    Uninstalling google-genai-1.16.1:
      Successfully uninstalled google-genai-1.16.1
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.95.0
    Uninstalling google-cloud-aiplatform-1.95.0:
      Successfully uninstalled google-cloud-aiplatform-1.95.0
  Attempting uninstall: google-adk
    Found existing installation: google-adk 0.5.0
    Uninstalling google-adk-0.5.0:
      Successfully uninstalled google-adk-0.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vertexai 1.71.1 requires google-cloud-aiplatform[all]==

# develop agent

In [3]:
! pip list | grep -e adk -e aiplatform

google-adk                               1.1.1
google-cloud-aiplatform                  1.71.1


In [4]:
!gsutil ls

gs://255766800726_us_central1_import_content/
gs://255766800726_us_central1_import_content_with_faq_csv/
gs://255766800726_us_import_content_with_faq_csv/
gs://255766800726_us_import_product/
gs://255766800726_us_import_user_event/
gs://avatar-bucket-notebook/
gs://cloud-ai-platform-67039ab0-1236-4b5e-8e7c-9bb8efa07266/
gs://dataproc-staging-us-central1-255766800726-grrfr5mh/
gs://genai-experiments-sm/
gs://hp-books-my-project-0004-346516-pytor-asia-southeast1/
gs://imagen-style-training/
gs://masked-audio-hk2/
gs://my-bk-my-project-0004--pytorch112kag-us-central1/
gs://my-my-project-0004--pytorch112kag-us-central1/
gs://my-project-0004-346516/
gs://my-project-0004-346516-asia-notebooks/
gs://my-project-0004-346516-darrylvideoragdemo-us-central1/
gs://my-project-0004-346516-jingletest1/
gs://my-project-0004-346516-movielens-data/
gs://my-project-0004-346516-pruclaim/
gs://my-project-0004-346516-sd/
gs://my-project-0004-346516-sd02/
gs://my-project-0004-346516-sd02-us-central1/
gs://my-

In [5]:
import vertexai
from google.adk.agents import Agent

In [6]:
import os

# Cloud project id.
PROJECT_IDS = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_IDS[0]  # @param {type:"string"}

if not PROJECT_ID:
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "us-central1" # @param {type:"string"}

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE" # Use Vertex AI API

# Print project and location details
print(f"Project ID:", PROJECT_ID)
print(f"Project Region:", LOCATION)


Project ID: my-project-0004-346516
Project Region: us-central1


In [7]:
from google.cloud import storage

client = storage.Client()

GCS_BUCKET_LOCATION = LOCATION
UNIQUE_PREFIX = 'sm' #"<REPLACE_WITH_PREFIX>"

GCS_BUCKET_NAME = f"{PROJECT_ID}-{UNIQUE_PREFIX}"
GCS_BUCKET_URI = f"gs://{GCS_BUCKET_NAME}"

bucket = storage.Bucket(client, GCS_BUCKET_NAME)

if bucket.exists()==False:
    # Create a Cloud Storage Bucket
    !gcloud storage buckets create $GCS_BUCKET_URI --location=$GCS_BUCKET_LOCATION

else:    
    print(f"\n{GCS_BUCKET_NAME} already exists.")
    
def gcs_file(blob_name):
    return bucket.blob(blob_name)



my-project-0004-346516-sm already exists.


In [8]:

STAGING_BUCKET = GCS_BUCKET_NAME  #@param{type:"string"}

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

In [ ]:
from google.adk.agents import Agent

root_agent = Agent(
    model='gemini-2.0-flash-001',
    name='hotel_agent',
    description='A helpful assistant that answers questions about a specific city.',
    instruction='Answer user questions about a specific city to the best of your knowledge. Do not answer questions outside of this.',
)

# local testing

In [ ]:
from vertexai.preview import reasoning_engines

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

In [ ]:
session = app.create_session(user_id="u_123")
session

Session(id='6ad75a29-0182-4db2-948b-69b4a4e350f3', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1748607443.148874)

In [ ]:
for event in app.stream_query(
    user_id="u_123",
    session_id=session.id,
    message="今日はいい天気ですね",
):
  print(event)

{'content': {'parts': [{'text': '今日は最悪の天気ですね。\n'}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 7, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 7}], 'prompt_token_count': 47, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 47}], 'total_token_count': 54, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-d72942ed-55ce-42b6-a794-79956d896df2', 'author': '天邪鬼', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'fy9Y3rnw', 'timestamp': 1748607444.218803}


# deploy agent

In [ ]:
from vertexai import agent_engines

name = dict(root_agent)['name']

# https://github.com/googleapis/python-aiplatform/blob/main/vertexai/agent_engines/_agent_engines.py
remote_app = agent_engines.create(
    display_name=name,
    agent_engine=root_agent,
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]",
    ]
)

INFO:vertexai.agent_engines:Deploying google.adk.agents.Agent as an application.
INFO:vertexai.agent_engines:Identified the following requirements: {'pydantic': '2.10.4', 'google-cloud-aiplatform': '1.95.1', 'cloudpickle': '3.1.0'}
INFO:vertexai.agent_engines:The following requirements are appended: {'cloudpickle==3.1.0', 'pydantic==2.10.4'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]', 'cloudpickle==3.1.0', 'pydantic==2.10.4']
INFO:vertexai.agent_engines:Using bucket yuyaono-dev1-bkt-agent-engine
INFO:vertexai.agent_engines:Wrote to gs://yuyaono-dev1-bkt-agent-engine/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://yuyaono-dev1-bkt-agent-engine/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://yuyaono-dev1-bkt-agent-engine/agent_engine/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
I

In [ ]:
for agent in agent_engines.list():
  print(agent)

resource name: projects/635273855015/locations/asia-northeast1/reasoningEngines/5711303199319654400


In [ ]:
print(remote_app.display_name)

天邪鬼


# remote testing

In [ ]:
remote_session = remote_app.create_session(user_id="u_456")
remote_session

FailedPrecondition: 400 Reasoning Engine Execution failed.
Please refer to our documentation (https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/troubleshooting/use) for checking logs and other troubleshooting tips.
Error Details: {"detail":"Agent Engine Error: An error occurred during invocation. Exception: Failed to create session.\nRequest Data: {'user_id': 'u_456'}"}

In [ ]:
remote_app.list_sessions(user_id="u_456")

In [ ]:
for event in remote_app.stream_query(
    user_id="u_456",
    session_id=remote_session["id"],
    message="今日は雨ですね。空が暗いです。",
):
    print(event)